<a href="https://colab.research.google.com/github/niallkehoe/FinPredict/blob/main/Machine_Learning_for_Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [2]:
allCompanies = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA", "TSLA", "GLNCY", "BHP", "RIO", "VALE", "CSUAY", "NGLOY", "ZIJMF", "WMT", "COST", "HD", "JD", "KR",  "WBA", "BABA", "TGT", "LOW", "LEN", "CAT", "CCS", "MTH", "TOL", "DHI", "KBH", "TM", "BYDDF", "MBGYY", "STLA", "GM", "RACE", "F", "HMC"]

techCompanies = ["AAPL","MSFT","AMZN","TSLA","NVDA", "PYPL"]
df = pd.read_csv(allCompanies[0]+".csv", index_col=0)
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df["Change"] = (df["Close"] - df["Open"]) / df["Open"] * 100

for i in range(1,len(allCompanies)):
  dfNew = pd.read_csv(allCompanies[i]+".csv", index_col=0)
  dfNew['Open'] = pd.to_numeric(dfNew['Open'], errors='coerce')
  dfNew["Change"] = (dfNew["Close"] - dfNew["Open"]) / dfNew["Open"] * 100
  df = df.merge(dfNew, on="Date", how = "inner", suffixes=["_"+allCompanies[i-1],"_"+allCompanies[i]])

columns = ["Change_"+a for a in allCompanies]
columns.append("Date")
dfPrices = df[columns]
dfPrices

,Change_AAPL,Change_AMZN,Change_GOOG,Change_META,Change_MSFT,Change_NVDA,Change_TSLA,Change_GLNCY,Change_BHP,Change_RIO,...,Change_KBH,Change_TM,Change_BYDDF,Change_MBGYY,Change_STLA,Change_GM,Change_RACE,Change_F,Change_HMC,Date
0,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
1,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
2,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
3,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
4,1.799608,2.734245,1.203489,1.652057,2.635408,0.940577,4.192046,NaN,2.187816,2.702297,...,NaN,NaN,NaN,NaN,0.405093,0.393038,NaN,0.838223,-0.077700,"Mar 13, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679,-2.371274,1.062770,1.596351,1.536204,3.274319,-0.559441,1.845280,-3.835616,-1.868577,-1.843677,...,-0.931232,-0.262613,-0.161031,-0.550711,0.099108,-0.640669,-3.473684,-0.317864,-0.029577,"Oct 26, 2015"
2680,2.056908,-3.011658,-3.518417,0.274752,1.089866,-0.139665,-2.721563,-1.086957,-0.592269,-0.648677,...,-0.285510,0.271132,0.160772,-0.547695,-1.076321,1.153630,-2.406093,-0.254615,0.059154,"Oct 23, 2015"
2681,1.049685,0.213220,0.804207,1.922487,1.051967,2.453102,0.070922,2.623907,0.927406,1.361816,...,-2.869139,0.400866,1.141925,0.948767,0.601805,-0.617284,-0.560715,0.581020,0.639659,"Oct 22, 2015"
2682,1.049685,0.213220,0.804207,1.922487,1.051967,2.453102,0.070922,2.623907,0.927406,1.361816,...,-2.869139,0.400866,1.141925,0.948767,0.601805,-0.617284,-0.560715,0.581020,0.639659,"Oct 22, 2015"


In [3]:
# remove outliers
# dfPrices = dfPrices.drop("Change_ZIJMF", axis=1)
# dfPrices = dfPrices.drop("Change_CSUA", axis=1)
dfPrices

,Change_AAPL,Change_AMZN,Change_GOOG,Change_META,Change_MSFT,Change_NVDA,Change_TSLA,Change_GLNCY,Change_BHP,Change_RIO,...,Change_KBH,Change_TM,Change_BYDDF,Change_MBGYY,Change_STLA,Change_GM,Change_RACE,Change_F,Change_HMC,Date
0,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
1,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
2,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
3,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
4,1.799608,2.734245,1.203489,1.652057,2.635408,0.940577,4.192046,NaN,2.187816,2.702297,...,NaN,NaN,NaN,NaN,0.405093,0.393038,NaN,0.838223,-0.077700,"Mar 13, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679,-2.371274,1.062770,1.596351,1.536204,3.274319,-0.559441,1.845280,-3.835616,-1.868577,-1.843677,...,-0.931232,-0.262613,-0.161031,-0.550711,0.099108,-0.640669,-3.473684,-0.317864,-0.029577,"Oct 26, 2015"
2680,2.056908,-3.011658,-3.518417,0.274752,1.089866,-0.139665,-2.721563,-1.086957,-0.592269,-0.648677,...,-0.285510,0.271132,0.160772,-0.547695,-1.076321,1.153630,-2.406093,-0.254615,0.059154,"Oct 23, 2015"
2681,1.049685,0.213220,0.804207,1.922487,1.051967,2.453102,0.070922,2.623907,0.927406,1.361816,...,-2.869139,0.400866,1.141925,0.948767,0.601805,-0.617284,-0.560715,0.581020,0.639659,"Oct 22, 2015"
2682,1.049685,0.213220,0.804207,1.922487,1.051967,2.453102,0.070922,2.623907,0.927406,1.361816,...,-2.869139,0.400866,1.141925,0.948767,0.601805,-0.617284,-0.560715,0.581020,0.639659,"Oct 22, 2015"


In [30]:
dfPrices.dropna().head(10)

,Change_AAPL,Change_AMZN,Change_GOOG,Change_META,Change_MSFT,Change_NVDA,Change_TSLA,Change_GLNCY,Change_BHP,Change_RIO,...,Change_KBH,Change_TM,Change_BYDDF,Change_MBGYY,Change_STLA,Change_GM,Change_RACE,Change_F,Change_HMC,Date
0,0.865944,1.119045,1.267863,3.433202,1.573515,2.413177,3.355705,0.450045,0.859788,1.004367,...,-0.729722,0.190602,0.396040,0.359527,0.228702,-2.358749,0.129494,-2.293202,0.713436,"Mar 14, 2023"
5,-1.138406,-2.093450,-1.610811,-0.828683,-0.991716,-1.896706,-0.964997,-0.798580,-0.988305,-1.516477,...,-3.187919,-0.883621,-3.018868,-0.409207,-1.565996,-3.129973,-1.708244,-2.654867,-0.830816,"Mar 10, 2023"
6,-1.934097,-1.526473,-1.936713,-2.500671,-1.368149,-3.056877,-4.066574,-0.889680,-1.619958,-1.827601,...,-0.084317,-1.184192,-0.546448,-0.660905,-1.480263,-4.301619,-1.294052,-4.377880,-1.382144,"Mar 09, 2023"
8,0.039264,0.341880,0.264831,1.148357,-0.133837,2.954826,-1.642888,1.148410,-0.031606,0.463158,...,0.650085,0.157717,1.056082,0.101112,0.054054,0.151134,0.405853,1.564945,0.113122,"Mar 08, 2023"
10,-1.366298,-0.542207,-1.309998,-2.375661,-0.838861,-1.322034,-1.917651,-3.424658,-2.185535,-2.228991,...,-0.510349,-1.557191,-0.889047,-1.987084,-1.915913,-1.193436,-1.780361,-0.850077,-1.234568,"Mar 07, 2023"
12,0.026009,-1.512764,1.292921,-1.648936,0.171587,-1.410573,-2.382391,-0.578035,-0.404355,-0.424541,...,-0.171184,0.014274,0.419287,-0.099108,-0.955921,-1.389566,-1.148080,-0.764526,-0.374532,"Mar 06, 2023"
13,2.019724,2.329092,1.380203,3.537894,1.229232,2.444254,1.534908,0.478469,0.258831,0.079872,...,0.196133,0.909288,2.473498,0.753390,0.426667,1.535033,-0.165502,2.830189,1.174687,"Mar 03, 2023"
14,1.059704,0.787660,2.726463,1.247244,1.849523,3.673070,2.227696,1.493776,2.442483,1.935306,...,1.705202,0.844843,0.139130,1.138127,1.325235,2.915907,0.861543,1.373183,0.928433,"Mar 02, 2023"
16,-1.035211,-1.811015,0.388199,-0.670141,-1.790557,-2.130045,-1.668202,0.900901,0.253968,0.636678,...,-0.739687,-0.095127,0.247875,-0.509684,-0.054735,0.103413,-1.121566,-0.242915,-0.266362,"Mar 01, 2023"
17,0.244815,1.170281,0.848783,1.768470,0.140523,-0.667465,-2.317299,0.338123,-0.489636,0.258807,...,-0.591883,-0.139593,1.240602,-0.621440,-0.736544,-1.198674,0.250270,0.000000,-0.192160,"Feb 28, 2023"


In [31]:
adjusted = dfPrices.dropna()


cols = adjusted.iloc[:, 2:]
cols = pd.concat([cols, adjusted.iloc[:, [0,-1]]], axis=1)
# cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
cols
# adjusted.iloc[:, 0:1]

train_df = pd.concat([adjusted.iloc[:, 1].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
# # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

# # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()
train_df.head(10)


,Change_AMZN,Change_GOOG,Change_META,Change_MSFT,Change_NVDA,Change_TSLA,Change_GLNCY,Change_BHP,Change_RIO,Change_VALE,...,Change_BYDDF,Change_MBGYY,Change_STLA,Change_GM,Change_RACE,Change_F,Change_HMC,Date,Change_AAPL,Date
5,1.119045,-1.610811,-0.828683,-0.991716,-1.896706,-0.964997,-0.798580,-0.988305,-1.516477,-1.159243,...,-3.018868,-0.409207,-1.565996,-3.129973,-1.708244,-2.654867,-0.830816,"Mar 10, 2023",-1.138406,"Mar 10, 2023"
6,-2.093450,-1.936713,-2.500671,-1.368149,-3.056877,-4.066574,-0.889680,-1.619958,-1.827601,-1.495215,...,-0.546448,-0.660905,-1.480263,-4.301619,-1.294052,-4.377880,-1.382144,"Mar 09, 2023",-1.934097,"Mar 09, 2023"
8,-1.526473,0.264831,1.148357,-0.133837,2.954826,-1.642888,1.148410,-0.031606,0.463158,0.601685,...,1.056082,0.101112,0.054054,0.151134,0.405853,1.564945,0.113122,"Mar 08, 2023",0.039264,"Mar 08, 2023"
10,0.341880,-1.309998,-2.375661,-0.838861,-1.322034,-1.917651,-3.424658,-2.185535,-2.228991,-2.275449,...,-0.889047,-1.987084,-1.915913,-1.193436,-1.780361,-0.850077,-1.234568,"Mar 07, 2023",-1.366298,"Mar 07, 2023"
12,-0.542207,1.292921,-1.648936,0.171587,-1.410573,-2.382391,-0.578035,-0.404355,-0.424541,-0.239091,...,0.419287,-0.099108,-0.955921,-1.389566,-1.148080,-0.764526,-0.374532,"Mar 06, 2023",0.026009,"Mar 06, 2023"
13,-1.512764,1.380203,3.537894,1.229232,2.444254,1.534908,0.478469,0.258831,0.079872,-0.465387,...,2.473498,0.753390,0.426667,1.535033,-0.165502,2.830189,1.174687,"Mar 03, 2023",2.019724,"Mar 03, 2023"
14,2.329092,2.726463,1.247244,1.849523,3.673070,2.227696,1.493776,2.442483,1.935306,-0.349650,...,0.139130,1.138127,1.325235,2.915907,0.861543,1.373183,0.928433,"Mar 02, 2023",1.059704,"Mar 02, 2023"
16,0.787660,0.388199,-0.670141,-1.790557,-2.130045,-1.668202,0.900901,0.253968,0.636678,2.485207,...,0.247875,-0.509684,-0.054735,0.103413,-1.121566,-0.242915,-0.266362,"Mar 01, 2023",-1.035211,"Mar 01, 2023"
17,-1.811015,0.848783,1.768470,0.140523,-0.667465,-2.317299,0.338123,-0.489636,0.258807,0.245399,...,1.240602,-0.621440,-0.736544,-1.198674,0.250270,0.000000,-0.192160,"Feb 28, 2023",0.244815,"Feb 28, 2023"
19,1.170281,-0.312395,1.037713,-0.296047,0.262648,0.280141,-0.341588,0.498919,-0.534142,-0.669100,...,-0.927247,-0.886802,1.093211,0.693909,0.547148,0.507614,-0.193798,"Feb 24, 2023",-0.271905,"Feb 24, 2023"


The trading strategy we use is going to be buying when the model predicts that the stocks will go up, and then selling at the end of the day. If the model predicts the stock to go down, we ignore the stock on this day, since Robinhood does not allow inexperienced traders to short stocks.

The graphs and "revenue change" metrics represent the total change from an initial bankroll of 1 unit. The graphs and revenue changes below show how the initial 1 unit bankrolls would have grown if someone was using the model to trade in the market. It was necessary to set the test set as the last 365 days, because otherwise, the revenue change metric would be somewhat nonsensical. Smaller and bigger test sizes were experimented with, but they did not make a big difference on the result. The metric of "Overall Revenue Growth" was chosen as a better metric than accuracy or F1 score (on whether the stock went up or down) or MSE, because it was more rooted in the actual result we are trying to optimize for. 

The results below are extremely good. AMZN in this time frame is down from 161 to 98 dollars per share. GOOG is down from 136 to 102 dollars per share. To be profitable, and especially to be THIS profitable, shows very high performance.

In [6]:
train_df.columns

Index(['Change_AMZN', 'Change_AAPL', 'Change_AMZN', 'Change_GOOG',
       'Change_META', 'Change_MSFT', 'Change_NVDA', 'Change_TSLA',
       'Change_GLNCY', 'Change_BHP', 'Change_RIO', 'Change_VALE',
       'Change_CSUAY', 'Change_NGLOY', 'Change_ZIJMF', 'Change_WMT',
       'Change_COST', 'Change_HD', 'Change_JD', 'Change_KR', 'Change_WBA',
       'Change_BABA', 'Change_TGT', 'Change_LOW', 'Change_LEN', 'Change_CAT',
       'Change_CCS', 'Change_MTH', 'Change_TOL', 'Change_DHI', 'Change_KBH',
       'Change_TM', 'Change_BYDDF', 'Change_MBGYY', 'Change_STLA', 'Change_GM',
       'Change_RACE', 'Change_F', 'Change_HMC'],
      dtype='object')

In [47]:
# @title Predicting GOOG Stock Change based on Previous-day Change (RF Regressor)
adjusted = dfPrices.dropna()


cols = adjusted.iloc[:, 4:]
cols = pd.concat([cols, adjusted.iloc[:, [0,1,2,-1]]], axis=1)
# cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
cols
# adjusted.iloc[:, 0:1]

train_df = pd.concat([adjusted.iloc[:, 3].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
# # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

# # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()

# # train_df = pd.concat([adjusted.iloc[:, -1].shift(1), adjusted.iloc[:, :-1]], axis=1)
# train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
# # train_df.loc[train_df["Change_AMZN"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AMZN"] <= 0, "positive"] = -3
# train_df = train_df.dropna()
# train_df

#testing on last 375 days

x = 238
train = train_df[:-x]
test = train_df[-x:]

model = RandomForestRegressor(n_estimators=100, random_state=42)

X_train = train.drop('Change_AMZN', axis=1)
X_train = X_train.drop('Date', axis=1)
# X_train = X_train.drop('positive', axis=1)
y_train = train['Change_AMZN']

model.fit(X_train, y_train)


X_test = test.drop('Change_GOOG', axis=1)
X_test = X_test.drop('Date', axis=1)
# X_test = X_test.drop('positive', axis=1)
y_test = test['Change_GOOG']
y_pred = model.predict(X_test)


pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

wrong = 0

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  if (revenue_changes[i] < 1):
    wrong+=1
  bankroll.append(var)

print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Overall revenue change: 151.69%


In [33]:
# @title Predicting AMZN Stock Change based on Previous-day Change (RF Regressor)
adjusted = dfPrices.dropna()


cols = adjusted.iloc[:, 2:]
cols = pd.concat([cols, adjusted.iloc[:, [0,-1]]], axis=1)
# cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
cols
# adjusted.iloc[:, 0:1]

train_df = pd.concat([adjusted.iloc[:, 1].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
# # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

# # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()

# # train_df = pd.concat([adjusted.iloc[:, -1].shift(1), adjusted.iloc[:, :-1]], axis=1)
# train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
# # train_df.loc[train_df["Change_AMZN"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AMZN"] <= 0, "positive"] = -3
# train_df = train_df.dropna()
# train_df

#testing on last 375 days

x = 238
train = train_df[:-x]
test = train_df[-x:]

model = RandomForestRegressor(n_estimators=100, random_state=42)

X_train = train.drop('Change_AMZN', axis=1)
X_train = X_train.drop('Date', axis=1)
# X_train = X_train.drop('positive', axis=1)
y_train = train['Change_AMZN']

model.fit(X_train, y_train)


X_test = test.drop('Change_AMZN', axis=1)
X_test = X_test.drop('Date', axis=1)
# X_test = X_test.drop('positive', axis=1)
y_test = test['Change_AMZN']
y_pred = model.predict(X_test)


pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

wrong = 0

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  if (revenue_changes[i] < 1):
    wrong+=1
  bankroll.append(var)

print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Overall revenue change: 0.29%


In [18]:
print(wrong)

33


In [34]:

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt

roll = [1]

for index, row in pvt.iterrows():
  if row["y_pred"] > 0:
    am = 1+(row["y_test"]/100)
    am *= roll[-1]
    roll.append(am)
  # else:
  #   am = 1-(row["y_test"]/100)
  #   am *= roll[-1]
  #   roll.append(am)

roll
# pvt["positive"] = (pvt["y_pred"] > 0) * 1 
# pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

# revenue_changes = pvt["bankroll_movement"].tolist()
# var = 1
# revenue_changes
# bankroll = []

# wrong = 0

# for i in range(len(revenue_changes)):
#   var *= revenue_changes[i]
#   if (revenue_changes[i] < 1):
#     wrong+=1
#   bankroll.append(var)

# print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(roll)

In [ ]:
# @title Predicting AMZN Stock Change based on Previous-day Change (Linear Regression)

train_df = pd.concat([dfPrices.iloc[:, -1].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df.loc[train_df["Change_AMZN"] > 0, "positive"] = 3
train_df.loc[train_df["Change_AMZN"] <= 0, "positive"] = -3
train_df = train_df.dropna()
train_df



# cross-validation

# X = train_df.drop('Change_AMZN', axis=1)
# X = X.drop('Date', axis=1)
# X = X.drop('positive', axis=1)
# y = train_df['Change_AMZN']

# model_full = RandomForestRegressor()
# scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
# print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


#testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = LinearRegression()

X_train = train.drop('Change_AMZN', axis=1)
X_train = X_train.drop('Date', axis=1)
X_train = X_train.drop('positive', axis=1)
y_train = train['Change_AMZN']

model.fit(X_train, y_train)


X_test = test.drop('Change_AMZN', axis=1)
X_test = X_test.drop('Date', axis=1)
X_test = X_test.drop('positive', axis=1)
y_test = test['Change_AMZN']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()

#@title Calculating profit

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Overall revenue change: 143.20%


In [ ]:
train_df = pd.concat([dfPrices.iloc[:, 2].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()
train_df

# @title Predicting GOOG Stock Change based on Previous-day Change (RF Regressor)

train_df = pd.concat([dfPrices.iloc[:, -1].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df.loc[train_df["Change_GOOG"] > 0, "positive"] = 3
train_df.loc[train_df["Change_GOOG"] <= 0, "positive"] = -3
train_df = train_df.dropna()
train_df



# cross-validation

# X = train_df.drop('Change_GOOG', axis=1)
# X = X.drop('Date', axis=1)
# X = X.drop('positive', axis=1)
# y = train_df['Change_GOOG']

# model_full = RandomForestRegressor()
# scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
# print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


#testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = RandomForestRegressor(n_estimators=100, random_state=42)

X_train = train.drop('Change_GOOG', axis=1)
X_train = X_train.drop('Date', axis=1)
X_train = X_train.drop('positive', axis=1)
y_train = train['Change_GOOG']

model.fit(X_train, y_train)


X_test = test.drop('Change_GOOG', axis=1)
X_test = X_test.drop('Date', axis=1)
X_test = X_test.drop('positive', axis=1)
y_test = test['Change_GOOG']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Cross Validated mean squared error: 0.787
Overall revenue change: 93.84%


In [ ]:
train_df = pd.concat([dfPrices.iloc[:, 2].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()
train_df

# @title Predicting GOOG Stock Change based on Previous-day Change (Linear Regressor)

train_df = pd.concat([dfPrices.iloc[:, -1].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df.loc[train_df["Change_GOOG"] > 0, "positive"] = 3
train_df.loc[train_df["Change_GOOG"] <= 0, "positive"] = -3
train_df = train_df.dropna()
train_df



# cross-validation

# X = train_df.drop('Change_GOOG', axis=1)
# X = X.drop('Date', axis=1)
# X = X.drop('positive', axis=1)
# y = train_df['Change_GOOG']

# model_full = RandomForestRegressor()
# scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
# print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


#testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = LinearRegression()

X_train = train.drop('Change_GOOG', axis=1)
X_train = X_train.drop('Date', axis=1)
X_train = X_train.drop('positive', axis=1)
y_train = train['Change_GOOG']

model.fit(X_train, y_train)


X_test = test.drop('Change_GOOG', axis=1)
X_test = X_test.drop('Date', axis=1)
X_test = X_test.drop('positive', axis=1)
y_test = test['Change_GOOG']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Overall revenue change: 96.20%


From the testing of these two companies, it is encouraging to see strong growth of our initial bankroll, especially since there do not seem to be big dips in the data. This tells us that the changes in stock can in fact be predicted by other From here, we should test different 

In [45]:
column_headers = list(dfPrices.columns.values)

for i in range(1, 38):
  # train_df = pd.concat([dfPrices.iloc[:, -i].shift(1), dfPrices.iloc[:, :-1]], axis=1)
  # train_df = train_df.dropna()
  # train_df


  cols = adjusted.iloc[:, i+1:]
  indexs = list(range(0, i))
  indexs.append(-1)
  cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
  # cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
  cols
  # adjusted.iloc[:, 0:1]

  train_df = pd.concat([adjusted.iloc[:, i].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
  # # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
  # # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

  # # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
  train_df = train_df.dropna()


  # cross-validation

  # X = train_df.drop('Change_AMZN', axis=1)
  # X = X.drop('Date', axis=1)
  # X = X.drop('positive', axis=1)
  # y = train_df['Change_AMZN']

  # model_full = RandomForestRegressor()
  # scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
  # print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


  #testing on last 375 days
  x = 238
  train = train_df[:-x]
  test = train_df[-x:]

  model = RandomForestRegressor(n_estimators=100, random_state=42)

  X_train = train.drop(column_headers[i], axis=1)
  try:
    X_train = X_train.drop('Date', axis=1)
  except:
    pass
  y_train = train[column_headers[i]]

  model.fit(X_train, y_train)


  X_test = test.drop(column_headers[i], axis=1)
  try:
    X_test = X_test.drop('Date', axis=1)
  except:
    pass
  y_test = test[column_headers[i]]
  y_pred = model.predict(X_test)



  # #plotting results
  # plt.plot(y_test.values, label='Actual')
  # plt.plot(y_pred, label='Predicted')
  # plt.ylabel("% Change in Day")
  # plt.legend()
  # plt.show()

  #@title Testing the Random Forest Regressor

  pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
  pvt["positive"] = (pvt["y_pred"] > 0) * 1 
  pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

  revenue_changes = pvt["bankroll_movement"].tolist()
  var = 1
  revenue_changes
  bankroll = []
  string1 = str(column_headers[i])
  for i in range(len(revenue_changes)):
    var *= revenue_changes[i]
    bankroll.append(var)
  print(string1 + f" Overall revenue change: {(var *100) - 100:.2f}%")

  # px.line(bankroll)

Change_AMZN Overall revenue change: 0.29%
Change_GOOG Overall revenue change: 17.87%
Change_META Overall revenue change: -6.10%
Change_MSFT Overall revenue change: -5.29%
Change_NVDA Overall revenue change: 38.51%
Change_TSLA Overall revenue change: -19.93%
Change_GLNCY Overall revenue change: 73.67%
Change_BHP Overall revenue change: 47.62%
Change_RIO Overall revenue change: 19.03%
Change_VALE Overall revenue change: 33.03%
Change_CSUAY Overall revenue change: 119.33%
Change_NGLOY Overall revenue change: 100.43%
Change_ZIJMF Overall revenue change: -16.27%
Change_WMT Overall revenue change: 11.29%
Change_COST Overall revenue change: 1.35%
Change_HD Overall revenue change: -2.69%
Change_JD Overall revenue change: -6.24%
Change_KR Overall revenue change: 17.25%
Change_WBA Overall revenue change: 7.56%
Change_BABA Overall revenue change: 7.81%
Change_TGT Overall revenue change: -8.41%
Change_LOW Overall revenue change: 2.72%
Change_LEN Overall revenue change: -27.75%
Change_CAT Overall r

In [54]:
column_headers = list(dfPrices.columns.values)

for i in range(1, 38):
  # train_df = pd.concat([dfPrices.iloc[:, -i].shift(1), dfPrices.iloc[:, :-1]], axis=1)
  # train_df = train_df.dropna()
  # train_df


  cols = adjusted.iloc[:, i+1:]
  indexs = list(range(0, i))
  indexs.append(-1)
  cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
  # cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
  cols
  # adjusted.iloc[:, 0:1]

  train_df = pd.concat([adjusted.iloc[:, i].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
  # # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
  # # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

  # # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
  train_df = train_df.dropna()

  # cross-validation

  # X = train_df.drop('Change_AMZN', axis=1)
  # X = X.drop('Date', axis=1)
  # X = X.drop('positive', axis=1)
  # y = train_df['Change_AMZN']

  # model_full = RandomForestRegressor()
  # scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
  # print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


  #testing on last 375 days
  x = 238
  train = train_df[:-x]
  test = train_df[-x:]

  model = LinearRegression()

  X_train = train.drop(column_headers[i], axis=1)
  try:
    X_train = X_train.drop('Date', axis=1)
  except:
    pass
  y_train = train[column_headers[i]]

  model.fit(X_train, y_train)


  X_test = test.drop(column_headers[i], axis=1)
  try:
    X_test = X_test.drop('Date', axis=1)
  except:
    pass
  y_test = test[column_headers[i]]
  y_pred = model.predict(X_test)



  # #plotting results
  # plt.plot(y_test.values, label='Actual')
  # plt.plot(y_pred, label='Predicted')
  # plt.ylabel("% Change in Day")
  # plt.legend()
  # plt.show()

  #@title Testing Linear Regression

  pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
  pvt["positive"] = (pvt["y_pred"] > 0) * 1 
  pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

  revenue_changes = pvt["bankroll_movement"].tolist()
  var = 1
  revenue_changes
  bankroll = []
  string1 = str(column_headers[i])
  for i in range(len(revenue_changes)):
    var *= revenue_changes[i]
    bankroll.append(var)

  print(string1 + f" Overall revenue change: {(var *100) - 100:.2f}%")

  # if string1 == "Change_CAT":
  #   px.line(bankroll)

Change_AMZN Overall revenue change: 1.75%
Change_GOOG Overall revenue change: 3.50%
Change_META Overall revenue change: 1.26%
Change_MSFT Overall revenue change: 15.90%
Change_NVDA Overall revenue change: 37.77%
Change_TSLA Overall revenue change: -34.58%
Change_GLNCY Overall revenue change: 45.43%
Change_BHP Overall revenue change: 37.40%
Change_RIO Overall revenue change: 20.18%
Change_VALE Overall revenue change: 21.62%
Change_CSUAY Overall revenue change: 92.29%
Change_NGLOY Overall revenue change: 96.39%
Change_ZIJMF Overall revenue change: 23.17%
Change_WMT Overall revenue change: 14.26%
Change_COST Overall revenue change: 6.53%
Change_HD Overall revenue change: -6.42%
Change_JD Overall revenue change: -23.23%
Change_KR Overall revenue change: -8.01%
Change_WBA Overall revenue change: -6.37%
Change_BABA Overall revenue change: -1.39%
Change_TGT Overall revenue change: -6.98%
Change_LOW Overall revenue change: -7.43%
Change_LEN Overall revenue change: -19.62%
Change_CAT Overall re

In [48]:
column_headers = list(dfPrices.columns.values)

for i in range(1, 38):
  cols = adjusted.iloc[:, i+1:]
  indexs = list(range(0, i))
  indexs.append(-1)
  cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
  # cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
  cols
  # adjusted.iloc[:, 0:1]

  train_df = pd.concat([adjusted.iloc[:, i].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
  # # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
  # # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

  # # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
  train_df = train_df.dropna()

  # cross-validation

  # X = train_df.drop('Change_AMZN', axis=1)
  # X = X.drop('Date', axis=1)
  # X = X.drop('positive', axis=1)
  # y = train_df['Change_AMZN']

  # model_full = RandomForestRegressor()
  # scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
  # print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


  #testing on last 375 days
  x = 238
  train = train_df[:-x]
  test = train_df[-x:]

  model = KNeighborsRegressor(20)

  X_train = train.drop(column_headers[i], axis=1)
  try:
    X_train = X_train.drop('Date', axis=1)
  except:
    pass
  y_train = train[column_headers[i]]

  model.fit(X_train, y_train)


  X_test = test.drop(column_headers[i], axis=1)
  try:
    X_test = X_test.drop('Date', axis=1)
  except:
    pass
  y_test = test[column_headers[i]]
  y_pred = model.predict(X_test)



  # #plotting results
  # plt.plot(y_test.values, label='Actual')
  # plt.plot(y_pred, label='Predicted')
  # plt.ylabel("% Change in Day")
  # plt.legend()
  # plt.show()

  #@title Testing KNearestNeighbors

  pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
  pvt["positive"] = (pvt["y_pred"] > 0) * 1 
  pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

  revenue_changes = pvt["bankroll_movement"].tolist()
  var = 1
  revenue_changes
  bankroll = []
  string1 = str(column_headers[i])
  for i in range(len(revenue_changes)):
    var *= revenue_changes[i]
    bankroll.append(var)

  print(string1 + f" Overall revenue change: {(var *100) - 100:.2f}%")

  # px.line(bankroll)

Change_AMZN Overall revenue change: 18.11%
Change_GOOG Overall revenue change: -0.89%
Change_META Overall revenue change: -17.82%
Change_MSFT Overall revenue change: -2.96%
Change_NVDA Overall revenue change: 29.62%
Change_TSLA Overall revenue change: -14.25%
Change_GLNCY Overall revenue change: 9.15%
Change_BHP Overall revenue change: 23.11%
Change_RIO Overall revenue change: 4.70%
Change_VALE Overall revenue change: -40.36%
Change_CSUAY Overall revenue change: 88.53%
Change_NGLOY Overall revenue change: 49.10%
Change_ZIJMF Overall revenue change: -42.19%
Change_WMT Overall revenue change: 16.11%
Change_COST Overall revenue change: -0.22%
Change_HD Overall revenue change: 3.34%
Change_JD Overall revenue change: -19.26%
Change_KR Overall revenue change: 11.15%
Change_WBA Overall revenue change: -10.06%
Change_BABA Overall revenue change: 8.98%
Change_TGT Overall revenue change: 16.68%
Change_LOW Overall revenue change: -0.77%
Change_LEN Overall revenue change: -19.88%
Change_CAT Overal

Linear Regression has higher average profit, but Random Forest had a lower variance. Neither was significantly better than the other.

Here are some other interesting companies to look at. Our model was extremely good at predicting VALE, to the point where it made a 9X return on it's initial bankroll.

HMC was the only stock that the model failed to have a positive yield on. This was actually unsurprising, because from the data exploration, it was evident that HMC had low correlation with the other stocks in the data set.

In [46]:
adjusted.iloc[:, 13]

0       0.628931
5       0.000000
6       0.000000
8       0.000000
10      0.636943
          ...   
2678    0.000000
2679    0.000000
2680    0.000000
2681    0.000000
2683    0.000000
Name: Change_ZIJMF, Length: 1859, dtype: float64

In [50]:

adjusted = dfPrices.dropna()


# @title Predicting VALE Stock Change based on Previous-day Change (Linear Regression)

# train_df = pd.concat([dfPrices.iloc[:, -10].shift(1), dfPrices.iloc[:, :-1]], axis=1)
# # train_df.loc[train_df["Change_VALE"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_VALE"] <= 0, "positive"] = -3
# train_df = train_df.dropna()
# train_df


# i=11
# cols = adjusted.iloc[:, i+1:]
# indexs = list(range(0, i))
# indexs.append(-1)
# cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
# # cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
# cols
# # adjusted.iloc[:, 0:1]

# train_df = pd.concat([adjusted.iloc[:, i].shift(1), cols], axis=1)
# train_df = train_df.dropna()


cols = adjusted.iloc[:, 13:]
indexs = list(range(0, 12))
indexs.append(-1)
cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
# cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
cols
# adjusted.iloc[:, 0:1]

train_df = pd.concat([adjusted.iloc[:, 12].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
# # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
# # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

# # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()
train_df.head(10)

# cross-validation

# X = train_df.drop('Change_AMZN', axis=1)
# X = X.drop('Date', axis=1)
# X = X.drop('positive', axis=1)
# y = train_df['Change_AMZN']

# model_full = RandomForestRegressor()
# scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
# print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


#testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = LinearRegression()

X_train = train.drop('Change_NGLOY', axis=1)
# X_train = X_train.drop('positive', axis=1)
X_train = X_train.drop('Date', axis=1)
y_train = train['Change_NGLOY']

model.fit(X_train, y_train)


X_test = test.drop('Change_NGLOY', axis=1)
# X_test = X_test.drop('positive', axis=1)
X_test = X_test.drop('Date', axis=1)
y_test = test['Change_NGLOY']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()

#@title Calculating profit

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

# print("VALE:")
print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

Overall revenue change: 96.39%


In [59]:
column_headers = list(dfPrices.columns.values)

i = 24

cols = adjusted.iloc[:, i+1:]
indexs = list(range(0, i))
indexs.append(-1)
cols = pd.concat([cols, adjusted.iloc[:, indexs]], axis=1)
  # cols = cols.merge(adjusted.iloc[:, [0,-1]], on="Date").dropna()
cols
  # adjusted.iloc[:, 0:1]

train_df = pd.concat([adjusted.iloc[:, i].shift(1), cols], axis=1) #doing this makes the "1" column (in this case, AMZN) drop down, such that it aligns with the previous day's data. 
  # # train_df.loc[train_df["Change_AAPL"] > 0, "positive"] = 3
  # # train_df.loc[train_df["Change_AAPL"] <= 0, "positive"] = -3

  # # train_df = pd.concat([adjusted.iloc[:, 1].shift(1), adjusted.iloc[:, :-1]], axis=1)
train_df = train_df.dropna()

  # cross-validation

  # X = train_df.drop('Change_AMZN', axis=1)
  # X = X.drop('Date', axis=1)
  # X = X.drop('positive', axis=1)
  # y = train_df['Change_AMZN']

  # model_full = RandomForestRegressor()
  # scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
  # print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


  #testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = RandomForestRegressor()

X_train = train.drop(column_headers[i], axis=1)
try:
  X_train = X_train.drop('Date', axis=1)
except:
  pass
y_train = train[column_headers[i]]

model.fit(X_train, y_train)


X_test = test.drop(column_headers[i], axis=1)
try:
  X_test = X_test.drop('Date', axis=1)
except:
  pass
y_test = test[column_headers[i]]
y_pred = model.predict(X_test)


pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []
string1 = str(column_headers[i])
for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print(string1 + f" Overall revenue change: {(var *100) - 100:.2f}%")
  
px.line(bankroll)


Change_CAT Overall revenue change: 38.98%


In [ ]:
dfPrices["Change_VALE"] #there are days where VALE goes down, but the line is pretty much straight up

0       0.441084
1       0.441084
2            NaN
3            NaN
4       0.312110
          ...   
2679   -2.335456
2680   -2.272727
2681    0.852878
2682    0.852878
2683   -0.879121
Name: Change_VALE, Length: 2684, dtype: float64

In [ ]:
# @title Predicting HMC Stock Change based on Previous-day Change (Linear Regression)

train_df = pd.concat([dfPrices.iloc[:, -10].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df.loc[train_df["Change_ZIJMF"] > 0, "positive"] = 3
train_df.loc[train_df["Change_ZIJMF"] <= 0, "positive"] = -3
train_df = train_df.dropna()
train_df



# cross-validation

# X = train_df.drop('Change_AMZN', axis=1)
# X = X.drop('Date', axis=1)
# X = X.drop('positive', axis=1)
# y = train_df['Change_AMZN']

# model_full = RandomForestRegressor()
# scores = cross_val_score(model_full, X, y, cv=5, scoring='neg_mean_squared_error')
# print(f"Cross Validated mean squared error: {np.sqrt(abs(scores.mean())):.3f}")


#testing on last 375 days
x = 238
train = train_df[:-x]
test = train_df[-x:]

model = LinearRegression()

X_train = train.drop('Change_ZIJMF', axis=1)
X_train = X_train.drop('positive', axis=1)
X_train = X_train.drop('Date', axis=1)
y_train = train['Change_ZIJMF']

model.fit(X_train, y_train)


X_test = test.drop('Change_ZIJMF', axis=1)
X_test = X_test.drop('positive', axis=1)
X_test = X_test.drop('Date', axis=1)
y_test = test['Change_ZIJMF']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()

#@title Calculating profit of HMC

pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print("ZIJMF:")
print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

ZIJMF:
Overall revenue change: -1.72%


In [ ]:
dfPrices["tech"] = (dfPrices["Change_AAPL"] + dfPrices["Change_AMZN"] + dfPrices["Change_GOOG"] + dfPrices["Change_MSFT"] + dfPrices["Change_META"] + dfPrices["Change_NVDA"] + dfPrices["Change_TSLA"]) / 7
train_df = pd.concat([dfPrices.iloc[:, -1].shift(1), dfPrices.iloc[:, :-1]], axis=1)
train_df.loc[train_df["tech"] > 0, "positive"] = 3
train_df.loc[train_df["tech"] <= 0, "positive"] = -3
train_df = train_df.dropna()
#@title Data Manipulation: Creating Index and shifting all rows up one except for the row that should be tested

<ipython-input-58-832852eef395>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#@title Calculating profit of a Tech Index

x = 238
train = train_df[:-x]
test = train_df[-x:]

model = LinearRegression()

X_train = train.drop('tech', axis=1)
X_train = X_train.drop('positive', axis=1)
X_train = X_train.drop('Date', axis=1)
y_train = train['tech']

model.fit(X_train, y_train)


X_test = test.drop('tech', axis=1)
X_test = X_test.drop('positive', axis=1)
X_test = X_test.drop('Date', axis=1)
y_test = test['tech']
y_pred = model.predict(X_test)



# #plotting results
# plt.plot(y_test.values, label='Actual')
# plt.plot(y_pred, label='Predicted')
# plt.ylabel("% Change in Day")
# plt.legend()
# plt.show()


pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
pvt["positive"] = (pvt["y_pred"] > 0) * 1 
pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

revenue_changes = pvt["bankroll_movement"].tolist()
var = 1
revenue_changes
bankroll = []

for i in range(len(revenue_changes)):
  var *= revenue_changes[i]
  bankroll.append(var)

print("YTD:")
print(f"Overall revenue change: {(var *100) - 100:.2f}%")

px.line(bankroll)

VALE:
Overall revenue change: 6.85%


In [ ]:
#@title Buying NASDAQ based on whether we think tech Index will go up or down
nasdaq = pd.read_csv("nasdaq.csv")

nasdaq['date'] = pd.to_datetime(nasdaq['date'], errors='coerce').dt.strftime('%m/%d/%Y') #.date
testNew = dfPrices[0:x][["Date"]]
testNew["y_pred"] = y_pred
testProcess = testNew[["y_pred", "date"]] #.set_index("DATE")
combo = testProcess.merge(nasdaq, on="date")
combo["Profit"] = (combo["_NDXD-close"] - combo["_NDXD-open"]) / combo["_NDXD-open"]
col = combo["Profit"]
start = 1
bank = []
for i in col:
  start *= (1 + i)
  bank.append(start)

plt.plot(bank, label='bankroll')
print(start)

In [ ]:
#@title Buying S&P500 Minerals and Mining Index based on price changes of mining stocks
miningCompanies = ["GLNCY", "BHP", "RIO", "VALE", "CSUAY", "NGLOY", "ZIJMF"]

mktcaps = [59.63, 150.26, 171.78, 73.11, 626.85, 44.64, 356.80]

sumT = 0
for i in mktcaps:
  sumT+=i
for i in range(0,len(mktcaps)):
  mktcaps[i]/=sumT
dfPrices["mining"] = (mktcaps[0]* dfPrices["Change_"+miningCompanies[0]])

for i in range(1,len(miningCompanies)):
  dfPrices["mining"] += (mktcaps[1]* dfPrices["Change_"+miningCompanies[i]])

testNew = dfPrices[0:x][["Date"]]
testNew["y_pred"] = y_pred
testNew['date'] = pd.to_datetime(testNew['Date'], errors='coerce').dt.strftime('%m/%d/%Y')
testProcess = testNew[["y_pred", "date"]]

miningIndex = pd.read_csv("miningIndex.csv")
miningIndex['date'] = pd.to_datetime(miningIndex['date'], errors='coerce').dt.strftime('%m/%d/%Y') #.date

combo = testProcess.merge(miningIndex, on="date")
combo["Profit"] = (combo["_5SP151040-close"] - combo["_5SP151040-open"]) / combo["_5SP151040-open"]
combo

col = combo["Profit"]
start = 1
bank = []
for i in col:
  start *= (1 + i)
  bank.append(start)

plt.plot(bank, label='bankroll')
print(start)